## Dependencies

In [28]:
import numpy as np
import pandas as pd

from utils import *

## I/O - Initialization

In [29]:
# load the feature dataset as a dataframe
channel_names =  ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "FT7", "FC3", "FCZ", "FC4", "FT8", "T3", "C3", "Cz", "C4", "T4", "TP7", "CP3", "CPz", "CP4", "TP8", "T5", "P3", "PZ", "P4", "T6", "O1", "Oz" , "O2"]

#channel subsets
hop1 = ["Cz","CP3","CPz","P3"]
hop2 = ["Cz","CP3","CPz","P3","FT7","FC3","FCZ","FC4","C4","CP4","TP8","T5","PZ","T6","O1","Oz","O2"]
csv_file = 'eeg_features.csv'
channel_list = hop2
dataset = data_loader(csv_file)
reduced_dataset = channel_selection(dataset, channel_list)
all_features = reduced_dataset.columns[:len(reduced_dataset.columns) - 1]

In [ ]:
#calculate accuracy for each channel
for channel in channel_names:
    print(channel)
    models = ['K-NN', 'GBC']
    dataset = data_loader(csv_file)
    reduced_dataset = channel_selection(dataset, [channel])
    data = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
    for model in models:
        model_training(data, model, stats=False, cm=False, verbose=True)

In [4]:
all_features = reduced_dataset.columns[:len(reduced_dataset.columns) - 1]
#result = incremental_training(dataset=dataset, channel_list=channel_list, feature_subset=all_features, models=['K-NN'], mode='feature', save=True)

## P-Value Thresholding

In [5]:
p_all, p_dict = p_value_thresholding(reduced_dataset, feature_subset=all_features)

/Users/kemalcankucuk/Documents/PAWS Work/eeg-attention/selection.py:77: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = stats.ttest_ind(


## Manual Feature Selection

In [6]:
#models = ['K-NN', 'GBC']
models = ['K-NN']
subset_1 = all_features

data = data_preparation(dataset=reduced_dataset, feature_subset=subset_1, pca=True)
for model in models:
    model_training(data, model, stats=False, cm=False, verbose=True)

Accuracy of K-NN classifier on training set: 0.79962894
Accuracy of K-NN classifier on test set: 0.66913580


## Hyperparameter Tuning


### SVM

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix 
 
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
print('Result of the best model on the test set: ', grid_predictions)
print(grid.best_params_) 

a = '''
besto_model = SVC(C=10, gamma=0.1, kernel='rbf')
model = besto_model
dataset = data_loader(csv_file)
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)

model.fit(X_train, y_train)
training_acc = model.score(X_train, y_train)
test_acc = model.score(X_test, y_test)
test_acc
'''

### KNN

In [30]:
from sklearn.model_selection import GridSearchCV 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix 
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings("ignore")
 
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
best_channel_list = ["Cz","CP3","CPz","P3","FT7","FC3","FCZ","FC4","C4","CP4","TP8","T5","PZ","T6","O1","Oz","O2"]
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
  
# defining parameter range 
k_range = list(range(2, 300))
param_grid = {'n_neighbors': k_range}  

grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
print('Result of the best model on the test set: ', grid_predictions)
print(grid.best_params_)

Fitting 5 folds for each of 298 candidates, totalling 1490 fits


/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:2

[CV 4/5] END .....................n_neighbors=2;, score=0.894 total time=   1.1s
[CV 2/5] END .....................n_neighbors=2;, score=0.890 total time=   1.1s
[CV 3/5] END .....................n_neighbors=2;, score=0.882 total time=   1.1s
[CV 1/5] END .....................n_neighbors=3;, score=0.909 total time=   1.1s
[CV 2/5] END .....................n_neighbors=3;, score=0.902 total time=   1.1s
[CV 5/5] END .....................n_neighbors=2;, score=0.884 total time=   1.2s
[CV 1/5] END .....................n_neighbors=2;, score=0.892 total time=   1.2s
[CV 3/5] END .....................n_neighbors=3;, score=0.896 total time=   1.1s
[CV 5/5] END .....................n_neighbors=3;, score=0.902 total time=   1.1s


/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:2

[CV 4/5] END .....................n_neighbors=3;, score=0.909 total time=   1.3s


...

[CV 4/5] END ...................n_neighbors=297;, score=0.740 total time=   1.6s
[CV 3/5] END ...................n_neighbors=297;, score=0.728 total time=   1.8s
[CV 5/5] END ...................n_neighbors=297;, score=0.737 total time=   1.6s
[CV 1/5] END ...................n_neighbors=297;, score=0.732 total time=   1.6s


/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:2

[CV 2/5] END ...................n_neighbors=298;, score=0.736 total time=   1.7s


/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


[CV 1/5] END ...................n_neighbors=298;, score=0.731 total time=   1.6s
[CV 3/5] END ...................n_neighbors=298;, score=0.728 total time=   1.6s
[CV 4/5] END ...................n_neighbors=298;, score=0.739 total time=   1.5s
[CV 5/5] END ...................n_neighbors=298;, score=0.737 total time=   1.5s
[CV 3/5] END ...................n_neighbors=299;, score=0.729 total time=   1.3s
[CV 2/5] END ...................n_neighbors=299;, score=0.737 total time=   1.4s
[CV 4/5] END ...................n_neighbors=299;, score=0.739 total time=   1.4s
[CV 1/5] END ...................n_neighbors=299;, score=0.732 total time=   1.3s
[CV 5/5] END ...................n_neighbors=299;, score=0.738 total time=   1.2s
Result of the best model on the test set:  [0 1 1 ... 0 1 1]
{'n_neighbors': 3}


### DTC

In [15]:
from sklearn.model_selection import GridSearchCV 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix 
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings("ignore")
 
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
  
# defining parameter range 
depths = list(range(1, 100))
param_grid = {'max_depth': depths}  

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
test_acc = grid.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)
print(grid.best_params_)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 2/5] END .......................max_depth=1;, score=0.716 total time=   0.0s
[CV 3/5] END .......................max_depth=1;, score=0.713 total time=   0.0s
[CV 1/5] END .......................max_depth=1;, score=0.701 total time=   0.0s
[CV 4/5] END .......................max_depth=1;, score=0.707 total time=   0.0s
[CV 5/5] END .......................max_depth=1;, score=0.713 total time=   0.0s
[CV 1/5] END .......................max_depth=2;, score=0.710 total time=   0.1s
[CV 3/5] END .......................max_depth=2;, score=0.718 total time=   0.1s
[CV 4/5] END .......................max_depth=2;, score=0.707 total time=   0.1s
[CV 2/5] END .......................max_depth=2;, score=0.724 total time=   0.1s
[CV 5/5] END .......................max_depth=2;, score=0.708 total time=   0.1s
[CV 1/5] END .......................max_depth=3;, score=0.736 total time=   0.1s
[CV 2/5] END .......................max_depth=3

### GBC 

In [22]:
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix 
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings("ignore")
 
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)
  
# defining parameter range 
a='''param_grid = {'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150]}'''

param_grid = {'learning_rate': [1.0],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150]}
              
grid = GridSearchCV(GradientBoostingClassifier(), param_grid, refit = True, verbose = 3, n_jobs= -1) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
test_acc = grid.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)
print(grid.best_params_)

Fitting 5 folds for each of 70 candidates, totalling 350 fits


/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning

[CV 1/5] END learning_rate=1.0, max_depth=5, n_estimators=50;, score=0.791 total time=  10.6s
[CV 3/5] END learning_rate=1.0, max_depth=5, n_estimators=50;, score=0.794 total time=  10.6s
[CV 2/5] END learning_rate=1.0, max_depth=5, n_estimators=50;, score=0.802 total time=  10.7s


/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END learning_rate=1.0, max_depth=5, n_estimators=50;, score=0.794 total time=  10.8s
[CV 4/5] END learning_rate=1.0, max_depth=5, n_estimators=50;, score=0.798 total time=  11.0s


...

[CV 3/5] END learning_rate=1.0, max_depth=14, n_estimators=115;, score=0.838 total time=  21.8s
[CV 4/5] END learning_rate=1.0, max_depth=14, n_estimators=115;, score=0.847 total time=  20.4s


/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END learning_rate=1.0, max_depth=14, n_estimators=115;, score=0.845 total time=  22.4s


/Users/kemalcankucuk/Documents/PAWS Work/paws/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END learning_rate=1.0, max_depth=14, n_estimators=130;, score=0.833 total time=  21.4s
[CV 2/5] END learning_rate=1.0, max_depth=14, n_estimators=130;, score=0.841 total time=  21.2s
[CV 4/5] END learning_rate=1.0, max_depth=14, n_estimators=130;, score=0.851 total time=  20.4s
[CV 3/5] END learning_rate=1.0, max_depth=14, n_estimators=130;, score=0.849 total time=  22.9s
[CV 5/5] END learning_rate=1.0, max_depth=14, n_estimators=130;, score=0.848 total time=  20.3s
[CV 1/5] END learning_rate=1.0, max_depth=14, n_estimators=150;, score=0.845 total time=  19.7s
[CV 2/5] END learning_rate=1.0, max_depth=14, n_estimators=150;, score=0.828 total time=  19.6s
[CV 4/5] END learning_rate=1.0, max_depth=14, n_estimators=150;, score=0.862 total time=  18.6s
[CV 3/5] END learning_rate=1.0, max_depth=14, n_estimators=150;, score=0.845 total time=  19.5s
[CV 5/5] END learning_rate=1.0, max_depth=14, n_estimators=150;, score=0.840 total time=  16.9s
Result of the best model on the test set

In [33]:
grid_predictions = grid.predict(X_test)
test_acc = grid.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)
print(grid.best_params_)

Result of the best model on the test set:  0.9237818181818181
{'n_neighbors': 3}


In [ ]:
dataset = data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = data_preparation(dataset=reduced_dataset, feature_subset=all_features)

model = 

In [27]:
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = grid.predict(X_test)
test_acc = grid.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)
print(grid.best_params_)
model = GradientBoostingClassifier(learning_rate= 0.01,  max_depth= 12, n_estimators= 500)
model.fit(X_train, y_train) 
  
# print how our model looks after hyper-parameter tuning 
#print(grid.best_estimator_) 
grid_predictions = model.predict(X_test)
test_acc = model.score(X_test, y_test)
print('Result of the best model on the test set: ', test_acc)

Result of the best model on the test set:  0.8603213844252163
{'learning_rate': 1.0, 'max_depth': 12, 'n_estimators': 80}
Result of the best model on the test set:  0.8331273176761433


## Yurteri's Method

p_better = []
for i in range(len(accuracies) - 1):
    delta = accuracies[i+1] - accuracies[i]
    if delta <= 0:
        continue
    else:
        
        p_better.append(p_all[i])

data = data_preparation(selected_channels=selected_channels, selected_labels=selected_labels, feature_subset=p_better)
for model in models:
    training, test = model_training(data, model, stats=False, cm=False, verbose=True)

import matplotlib.pyplot as plt
plt.plot(np.linspace(1,len(accuracies),len(accuracies)), accuracies)
plt.legend()
plt.savefig('foo.png', bbox_inches='tight')
plt.grid()
plt.show()
plt.legend(['GBC', 'K-NN', 'SVM', 'DTC', 'NN'])